In [76]:
import pandas as pd
import numpy as np
from datetime import datetime

# Acquire Data


In [77]:
# bring in sample trade log from site
trades = pd.read_csv('trades_sample.csv')

In [78]:
# view df and all columns
pd.set_option('display.max_columns', None)
trades 

,Unnamed: 0,account_type,action,activity_code,amount,as_of_date,check_number,commission,commission_in_gross,contra_broker,cusip,description,effective_date,entry_date,exchange_rate,fees,from_location,front_exec,held_currency_code,interest,maturity_date,misc_fee,nfa_fee,order_number,output_currency_code,payee,price,principal,quantity,sales_credit,sec_fee,security_class_code,security_type,state_tax,symbol,tax_exempt_status,taxes,to_location,trade,transaction_date,user_defined_action
0,0,INVENTORY,Sale,NaN,158632.00,NaN,NaN,0,Y,NaN,01639PBJ9,ALISO VIEJO CALIF CTFS PARTN COPS ...,"Thu, 03 Sep 2020 12:00:00 am EDT","Tue, 01 Sep 2020 2:23:10 am EDT",1.0,0,NaN,NaN,USD,1525.00,NaN,0,0,N0001F99,USD,NaN,104.738,-157107.00,-150000,0,0,NaN,FIXED,0,FIXED 01639PBJ9,NaN,0,NaN,Y,"Tue, 01 Sep 2020 12:00:00 am EDT",NaN
1,1,INVENTORY,Purchase,NaN,-84760.04,NaN,NaN,0,N,NaN,64580CFQ5,NEW JERSEY INFRASTRUCTURE BK ENVIRONMENT REF B...,"Thu, 03 Sep 2020 12:00:00 am EDT","Tue, 01 Sep 2020 4:36:57 am EDT",1.0,0,NaN,VSIN,USD,19.44,NaN,0,0,N0007F99,USD,NaN,121.058,84740.60,70000,0,0,NaN,FIXED,0,FIXED 64580CFQ5,NaN,0,NaN,Y,"Tue, 01 Sep 2020 12:00:00 am EDT",NaN
2,2,INVENTORY,Sale,NaN,85589.47,NaN,NaN,0,N,NaN,64580CFQ5,NEW JERSEY INFRASTRUCTURE BK ENVIRONMENT REF B...,"Fri, 04 Sep 2020 12:00:00 am EDT","Wed, 02 Sep 2020 9:10:24 am EDT",1.0,0,NaN,NaN,USD,29.17,NaN,0,0,N0201F99,USD,NaN,122.229,-85560.30,-70000,0,0,NaN,FIXED,0,FIXED 64580CFQ5,NaN,0,NaN,Y,"Wed, 02 Sep 2020 12:00:00 am EDT",NaN
3,3,INVENTORY,Sale,NaN,125835.30,NaN,NaN,0,Y,NaN,054231QS7,AVON OHIO LOC SCH DIST GO UNLTD TAX IMPT BDS ...,"Thu, 10 Sep 2020 12:00:00 am EDT","Wed, 02 Sep 2020 1:19:33 am EDT",1.0,0,NaN,NaN,USD,0.00,NaN,0,0,N0401F99,USD,NaN,109.422,-125835.30,-115000,0,0,NaN,FIXED,0,FIXED 054231QS7,NaN,0,NaN,Y,"Wed, 02 Sep 2020 12:00:00 am EDT",NaN
4,4,INVENTORY,Purchase,NaN,-367239.00,NaN,NaN,0,Y,NaN,355525FA9,FRASER VY MET RECREATIONAL DIS REF,"Wed, 16 Sep 2020 12:00:00 am EDT","Wed, 02 Sep 2020 10:21:47 am EDT",1.0,0,NaN,NaN,USD,0.00,NaN,0,0,N0303F99,USD,NaN,122.413,367239.00,300000,0,0,NaN,FIXED,0,FIXED 355525FA9,NaN,0,NaN,Y,"Wed, 02 Sep 2020 12:00:00 am EDT",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
350,350,INVENTORY,Sale,NaN,68507.60,NaN,NaN,0,Y,NaN,368414AV0,GEAUGA CNTY OHIO SPL OBLIG SAL BDS ...,"Tue, 20 Jul 2021 12:00:00 am EDT","Fri, 16 Jul 2021 4:58:20 am EDT",1.0,0,NaN,NaN,USD,315.00,NaN,0,0,N0001F99,USD,NaN,97.418,-68192.60,-70000,0,0,NaN,FIXED,0,FIXED 368414AV0,NaN,0,NaN,Y,"Fri, 16 Jul 2021 12:00:00 am EDT",NaN
351,351,INVENTORY,Sale,NaN,73839.69,NaN,NaN,0,N,NaN,800851TV9,SANGER CALIF UNI SCH DIST ELECTION OF 2018 GO ...,"Tue, 20 Jul 2021 12:00:00 am EDT","Fri, 16 Jul 2021 9:11:58 am EDT",1.0,0,NaN,VSIN,USD,185.94,NaN,0,0,N0003F99,USD,NaN,98.205,-73653.75,-75000,0,0,NaN,FIXED,0,FIXED 800851TV9,NaN,0,NaN,Y,"Fri, 16 Jul 2021 12:00:00 am EDT",NaN
352,352,INVENTORY,Sale,NaN,73873.44,NaN,NaN,0,N,NaN,800851TV9,SANGER CALIF UNI SCH DIST ELECTION OF 2018 GO ...,"Tue, 20 Jul 2021 12:00:00 am EDT","Fri, 16 Jul 2021 1:06:58 pm EDT",1.0,0,NaN,NaN,USD,185.94,NaN,0,0,N0004F99,USD,NaN,98.250,-73687.50,-75000,0,0,NaN,FIXED,0,FIXED 800851TV9,NaN,0,NaN,Y,"Fri, 16 Jul 2021 12:00:00 am EDT",NaN
353,353,INVENTORY,Sale,NaN,111805.67,NaN,NaN,0,N,NaN,018112SY9,ALLEN TEX WTRWKS & SWR REV SYS BDS ...,"Thu, 29 Jul 2021 12:00:00 am EDT","Tue, 27 Jul 2021 7:40:51 am EDT",1.0,0,NaN,NaN,USD,241.67,NaN,0,0,N0001F99,USD,NaN,111.564,-111564.00,-100000,0,0,NaN,FIXED,0,FIXED 018112SY9,NaN,0,NaN,Y,"Tue, 27 Jul 2021 12:00:00 am EDT",NaN


# Prepare Data

In [79]:
# Drop unnecessary columns
trades = trades.drop(columns=['Unnamed: 0', 'amount', 'account_type', 'activity_code', 'as_of_date', 'check_number', 'commission', 'commission_in_gross', 'contra_broker', 'exchange_rate', 'fees', 'from_location', 'front_exec', 'held_currency_code', 'interest', 'maturity_date', 'misc_fee', 'nfa_fee', 'order_number', 'output_currency_code', 'payee', 'principal', 'sales_credit', 'sec_fee', 'security_class_code', 'security_type', 'state_tax', 'symbol', 'tax_exempt_status', 'taxes', 'to_location', 'trade',  'user_defined_action'])

In [80]:
# Divide trades by 1000 per bond quantity
trades['quantity'] = (trades.quantity)/1000

In [81]:
trades.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 355 entries, 0 to 354
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   action            355 non-null    object 
 1   cusip             355 non-null    object 
 2   description       355 non-null    object 
 3   effective_date    355 non-null    object 
 4   entry_date        355 non-null    object 
 5   price             355 non-null    float64
 6   quantity          355 non-null    float64
 7   transaction_date  355 non-null    object 
dtypes: float64(2), object(6)
memory usage: 22.3+ KB


In [82]:
def convert_date(feature):
    '''takes date/time object, drops timestamp, converts to datetime mm/dd/yyy format'''
    # use regex to drop leading 5 characters (e.g. Thu, )
    trades[feature] = trades[feature].str.replace(r'^(.{5})','')
    # use regex to drop timestamp section at end of string
    trades[feature] = trades[feature].str.replace(r'(.{15})\.*$','')
    # convert object to datetime format
    trades[feature] = pd.to_datetime(trades[feature])
    # format to mm/dd/yyyy format
    trades[feature] = trades[feature].apply(lambda x: x.strftime("%m/%d/%Y"))
    return trades[feature]

In [83]:
trades['transaction_date'] = convert_date('transaction_date')
trades['entry_date'] = convert_date('entry_date')
trades['effective_date'] = convert_date('effective_date')

<ipython-input-82-7ba02226cc12>:4: FutureWarning: The default value of regex will change from True to False in a future version.
  trades[feature] = trades[feature].str.replace(r'^(.{5})','')
<ipython-input-82-7ba02226cc12>:6: FutureWarning: The default value of regex will change from True to False in a future version.
  trades[feature] = trades[feature].str.replace(r'(.{15})\.*$','')


In [84]:
trades.head()

,action,cusip,description,effective_date,entry_date,price,quantity,transaction_date
0,Sale,01639PBJ9,ALISO VIEJO CALIF CTFS PARTN COPS ...,09/03/2020,09/01/2020,104.738,-150.0,09/01/2020
1,Purchase,64580CFQ5,NEW JERSEY INFRASTRUCTURE BK ENVIRONMENT REF B...,09/03/2020,09/01/2020,121.058,70.0,09/01/2020
2,Sale,64580CFQ5,NEW JERSEY INFRASTRUCTURE BK ENVIRONMENT REF B...,09/04/2020,09/02/2020,122.229,-70.0,09/02/2020
3,Sale,054231QS7,AVON OHIO LOC SCH DIST GO UNLTD TAX IMPT BDS ...,09/10/2020,09/02/2020,109.422,-115.0,09/02/2020
4,Purchase,355525FA9,FRASER VY MET RECREATIONAL DIS REF,09/16/2020,09/02/2020,122.413,300.0,09/02/2020


# Explore Data - Discrepencies/Anomalies

In [85]:
trades[trades.transaction_date != trades.entry_date]

,action,cusip,description,effective_date,entry_date,price,quantity,transaction_date
27,Purchase Cancellation,429326N23,HIDALGO CNTY TEX CTFS OBLIG 2020,10/14/2020,10/16/2020,117.717,-150.0,09/15/2020
28,Purchase Cancellation,429326N23,HIDALGO CNTY TEX CTFS OBLIG 2020,10/14/2020,10/16/2020,117.717,-150.0,09/15/2020
29,Purchase,429326N23,HIDALGO CNTY TEX CTFS OBLIG 2020,10/14/2020,10/16/2020,117.717,150.0,09/15/2020
30,Purchase,429326N23,HIDALGO CNTY TEX CTFS OBLIG 2020,10/14/2020,10/16/2020,117.717,150.0,09/15/2020
34,Purchase Cancellation,4140187X3,HARRIS CNTY TEX FLOOD CTL DIST IMPT REF BDS ...,09/30/2020,09/23/2020,121.355,-65.0,09/18/2020
35,Purchase,4140187X3,HARRIS CNTY TEX FLOOD CTL DIST IMPT REF BDS ...,09/30/2020,09/23/2020,121.355,65.0,09/18/2020
36,Purchase Cancellation,539770LM7,LOCKHART TEX INDPT SCH DIST TAXABLE UNLTD TAX ...,10/15/2020,10/20/2020,100.000,-125.0,09/18/2020
37,Purchase,539770LM7,LOCKHART TEX INDPT SCH DIST TAXABLE UNLTD TAX ...,10/15/2020,10/20/2020,100.000,125.0,09/18/2020
58,Sale Cancellation,539770LM7,LOCKHART TEX INDPT SCH DIST TAXABLE UNLTD TAX ...,10/15/2020,10/14/2020,99.300,125.0,10/05/2020
77,Purchase,198432KU9,COLUMBIA SC GENERAL OBLIGATION 3.00% MATURITY ...,10/19/2020,10/16/2020,102.762,40.0,10/15/2020


In [86]:
trades[trades.transaction_date != trades.entry_date]

,action,cusip,description,effective_date,entry_date,price,quantity,transaction_date
27,Purchase Cancellation,429326N23,HIDALGO CNTY TEX CTFS OBLIG 2020,10/14/2020,10/16/2020,117.717,-150.0,09/15/2020
28,Purchase Cancellation,429326N23,HIDALGO CNTY TEX CTFS OBLIG 2020,10/14/2020,10/16/2020,117.717,-150.0,09/15/2020
29,Purchase,429326N23,HIDALGO CNTY TEX CTFS OBLIG 2020,10/14/2020,10/16/2020,117.717,150.0,09/15/2020
30,Purchase,429326N23,HIDALGO CNTY TEX CTFS OBLIG 2020,10/14/2020,10/16/2020,117.717,150.0,09/15/2020
34,Purchase Cancellation,4140187X3,HARRIS CNTY TEX FLOOD CTL DIST IMPT REF BDS ...,09/30/2020,09/23/2020,121.355,-65.0,09/18/2020
35,Purchase,4140187X3,HARRIS CNTY TEX FLOOD CTL DIST IMPT REF BDS ...,09/30/2020,09/23/2020,121.355,65.0,09/18/2020
36,Purchase Cancellation,539770LM7,LOCKHART TEX INDPT SCH DIST TAXABLE UNLTD TAX ...,10/15/2020,10/20/2020,100.000,-125.0,09/18/2020
37,Purchase,539770LM7,LOCKHART TEX INDPT SCH DIST TAXABLE UNLTD TAX ...,10/15/2020,10/20/2020,100.000,125.0,09/18/2020
58,Sale Cancellation,539770LM7,LOCKHART TEX INDPT SCH DIST TAXABLE UNLTD TAX ...,10/15/2020,10/14/2020,99.300,125.0,10/05/2020
77,Purchase,198432KU9,COLUMBIA SC GENERAL OBLIGATION 3.00% MATURITY ...,10/19/2020,10/16/2020,102.762,40.0,10/15/2020


In [87]:
# sale cancellations
trades[trades.action == 'Sale Cancellation']

,action,cusip,description,effective_date,entry_date,price,quantity,transaction_date
41,Sale Cancellation,727866AH8,PLATTEVILLE-GILCREST FIRE PROT COPS ...,09/25/2020,09/23/2020,116.193,205.0,09/23/2020
54,Sale Cancellation,539770LM7,LOCKHART TEX INDPT SCH DIST TAXABLE UNLTD TAX ...,10/15/2020,10/05/2020,99.300,100.0,10/05/2020
58,Sale Cancellation,539770LM7,LOCKHART TEX INDPT SCH DIST TAXABLE UNLTD TAX ...,10/15/2020,10/14/2020,99.300,125.0,10/05/2020
173,Sale Cancellation,266669BY5,DURHAM CALIF UNI SCH DIST GO 2018 ELECTION BDS...,01/20/2021,01/14/2021,121.654,250.0,01/14/2021
182,Sale Cancellation,13063DLW6,CALIFORNIA ST VARIOUS PURP GO REF BDS,01/25/2021,01/25/2021,129.352,390.0,01/21/2021
274,Sale Cancellation,725464CT6,PITTSFIELD MASS ST QUALIFIED MUN PURP LN 2021,05/14/2021,05/14/2021,98.190,570.0,05/12/2021
283,Sale Cancellation,60787PCR6,MOFFAT CNTY COLO CTFS PARTN,05/27/2021,05/18/2021,118.341,185.0,05/18/2021
295,Sale Cancellation,916510AZ3,UPPER SAN JUAN COLO HEALTH SVC REF,05/28/2021,05/26/2021,111.250,30.0,05/26/2021
319,Sale Cancellation,244127W77,DEER PARK TEX INDPT SCH DIST UNLTD TAX BLDG BD...,06/29/2021,06/24/2021,110.405,250.0,06/24/2021
348,Sale Cancellation,242469EZ3,DEARBORN HEIGHTS MICH LTD TAX GO CAP IMPT BDS ...,07/15/2021,07/15/2021,110.767,150.0,07/13/2021


In [88]:
# purchase cancellations
trades[trades.action == 'Purchase Cancellation']

,action,cusip,description,effective_date,entry_date,price,quantity,transaction_date
19,Purchase Cancellation,429326M57,HIDALGO CNTY TEX CTFS OBLIG 2020,10/14/2020,09/15/2020,129.436,-200.0,09/15/2020
20,Purchase Cancellation,429326N23,HIDALGO CNTY TEX CTFS OBLIG 2020,10/14/2020,09/15/2020,117.717,-150.0,09/15/2020
21,Purchase Cancellation,429326M57,HIDALGO CNTY TEX CTFS OBLIG 2020,10/14/2020,09/15/2020,129.436,-200.0,09/15/2020
27,Purchase Cancellation,429326N23,HIDALGO CNTY TEX CTFS OBLIG 2020,10/14/2020,10/16/2020,117.717,-150.0,09/15/2020
28,Purchase Cancellation,429326N23,HIDALGO CNTY TEX CTFS OBLIG 2020,10/14/2020,10/16/2020,117.717,-150.0,09/15/2020
34,Purchase Cancellation,4140187X3,HARRIS CNTY TEX FLOOD CTL DIST IMPT REF BDS ...,09/30/2020,09/23/2020,121.355,-65.0,09/18/2020
36,Purchase Cancellation,539770LM7,LOCKHART TEX INDPT SCH DIST TAXABLE UNLTD TAX ...,10/15/2020,10/20/2020,100.000,-125.0,09/18/2020
131,Purchase Cancellation,189184HQ0,CLOVERDALE CALIF UN SCH DIST ELECTION 2018 GO ...,12/07/2020,12/03/2020,115.750,-1000.0,12/03/2020
161,Purchase Cancellation,801577KZ9,SANTA CLARA CNTY CALIF FING AU REF LEASE REV B...,01/08/2021,01/06/2021,113.097,-200.0,01/06/2021
226,Purchase Cancellation,64966QVR8,NEW YORK N Y GO BDS FISCA,03/24/2021,03/09/2021,104.913,-500.0,03/09/2021


In [89]:
trades['effective_date'] = pd.to_datetime(trades.effective_date)
trades['transaction_date'] = pd.to_datetime(trades.transaction_date)
trades['entry_date'] = pd.to_datetime(trades.entry_date)

In [90]:
trades.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 355 entries, 0 to 354
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   action            355 non-null    object        
 1   cusip             355 non-null    object        
 2   description       355 non-null    object        
 3   effective_date    355 non-null    datetime64[ns]
 4   entry_date        355 non-null    datetime64[ns]
 5   price             355 non-null    float64       
 6   quantity          355 non-null    float64       
 7   transaction_date  355 non-null    datetime64[ns]
dtypes: datetime64[ns](3), float64(2), object(3)
memory usage: 22.3+ KB


In [91]:
trades.head()

,action,cusip,description,effective_date,entry_date,price,quantity,transaction_date
0,Sale,01639PBJ9,ALISO VIEJO CALIF CTFS PARTN COPS ...,2020-09-03,2020-09-01,104.738,-150.0,2020-09-01
1,Purchase,64580CFQ5,NEW JERSEY INFRASTRUCTURE BK ENVIRONMENT REF B...,2020-09-03,2020-09-01,121.058,70.0,2020-09-01
2,Sale,64580CFQ5,NEW JERSEY INFRASTRUCTURE BK ENVIRONMENT REF B...,2020-09-04,2020-09-02,122.229,-70.0,2020-09-02
3,Sale,054231QS7,AVON OHIO LOC SCH DIST GO UNLTD TAX IMPT BDS ...,2020-09-10,2020-09-02,109.422,-115.0,2020-09-02
4,Purchase,355525FA9,FRASER VY MET RECREATIONAL DIS REF,2020-09-16,2020-09-02,122.413,300.0,2020-09-02


In [92]:
trades.groupby(['cusip', 'effective_date']).head()

,action,cusip,description,effective_date,entry_date,price,quantity,transaction_date
0,Sale,01639PBJ9,ALISO VIEJO CALIF CTFS PARTN COPS ...,2020-09-03,2020-09-01,104.738,-150.0,2020-09-01
1,Purchase,64580CFQ5,NEW JERSEY INFRASTRUCTURE BK ENVIRONMENT REF B...,2020-09-03,2020-09-01,121.058,70.0,2020-09-01
2,Sale,64580CFQ5,NEW JERSEY INFRASTRUCTURE BK ENVIRONMENT REF B...,2020-09-04,2020-09-02,122.229,-70.0,2020-09-02
3,Sale,054231QS7,AVON OHIO LOC SCH DIST GO UNLTD TAX IMPT BDS ...,2020-09-10,2020-09-02,109.422,-115.0,2020-09-02
4,Purchase,355525FA9,FRASER VY MET RECREATIONAL DIS REF,2020-09-16,2020-09-02,122.413,300.0,2020-09-02
...,...,...,...,...,...,...,...,...
350,Sale,368414AV0,GEAUGA CNTY OHIO SPL OBLIG SAL BDS ...,2021-07-20,2021-07-16,97.418,-70.0,2021-07-16
351,Sale,800851TV9,SANGER CALIF UNI SCH DIST ELECTION OF 2018 GO ...,2021-07-20,2021-07-16,98.205,-75.0,2021-07-16
352,Sale,800851TV9,SANGER CALIF UNI SCH DIST ELECTION OF 2018 GO ...,2021-07-20,2021-07-16,98.250,-75.0,2021-07-16
353,Sale,018112SY9,ALLEN TEX WTRWKS & SWR REV SYS BDS ...,2021-07-29,2021-07-27,111.564,-100.0,2021-07-27


In [93]:
# trades by trade date
trades.sort_values(by=['cusip', 'entry_date'])

,action,cusip,description,effective_date,entry_date,price,quantity,transaction_date
88,Purchase,014393YU0,ALDINE TEX INDPT SCH DIST UNLTD TAX REF BDS ...,2020-11-19,2020-10-23,112.067,1000.0,2020-10-23
93,Sale,014393YU0,ALDINE TEX INDPT SCH DIST UNLTD TAX REF BDS ...,2020-11-19,2020-10-28,113.272,-1000.0,2020-10-28
158,Purchase,016059HU4,ALHAMBRA CALIF UNI SCH DIST GO BDS ...,2021-01-07,2021-01-05,127.540,100.0,2021-01-05
166,Sale,016059HU4,ALHAMBRA CALIF UNI SCH DIST GO BDS ...,2021-01-13,2021-01-11,127.689,-75.0,2021-01-11
201,Sale,016059HU4,ALHAMBRA CALIF UNI SCH DIST GO BDS ...,2021-02-16,2021-02-11,127.428,-25.0,2021-02-11
...,...,...,...,...,...,...,...,...
322,Sale,956510BT7,WEST VA PKWYS AUTH TPK TOLL RE SR LIEN BDS ...,2021-07-01,2021-06-29,131.645,-100.0,2021-06-29
102,Purchase,968546MF5,WILKINSBURG-PENN PA JT WTR AUT BDS ...,2020-11-06,2020-11-04,120.692,35.0,2020-11-04
107,Sale,968546MF5,WILKINSBURG-PENN PA JT WTR AUT BDS ...,2020-11-10,2020-11-06,122.500,-35.0,2020-11-06
115,Purchase,977810KA7,WOLFFORTH TEX TAX WTRWKS SWR SYS REV C 2021,2021-01-05,2020-11-17,122.096,315.0,2020-11-17


In [94]:
# trades by settlement
trades.sort_values(by=['cusip', 'effective_date'])

,action,cusip,description,effective_date,entry_date,price,quantity,transaction_date
88,Purchase,014393YU0,ALDINE TEX INDPT SCH DIST UNLTD TAX REF BDS ...,2020-11-19,2020-10-23,112.067,1000.0,2020-10-23
93,Sale,014393YU0,ALDINE TEX INDPT SCH DIST UNLTD TAX REF BDS ...,2020-11-19,2020-10-28,113.272,-1000.0,2020-10-28
158,Purchase,016059HU4,ALHAMBRA CALIF UNI SCH DIST GO BDS ...,2021-01-07,2021-01-05,127.540,100.0,2021-01-05
166,Sale,016059HU4,ALHAMBRA CALIF UNI SCH DIST GO BDS ...,2021-01-13,2021-01-11,127.689,-75.0,2021-01-11
201,Sale,016059HU4,ALHAMBRA CALIF UNI SCH DIST GO BDS ...,2021-02-16,2021-02-11,127.428,-25.0,2021-02-11
...,...,...,...,...,...,...,...,...
322,Sale,956510BT7,WEST VA PKWYS AUTH TPK TOLL RE SR LIEN BDS ...,2021-07-01,2021-06-29,131.645,-100.0,2021-06-29
102,Purchase,968546MF5,WILKINSBURG-PENN PA JT WTR AUT BDS ...,2020-11-06,2020-11-04,120.692,35.0,2020-11-04
107,Sale,968546MF5,WILKINSBURG-PENN PA JT WTR AUT BDS ...,2020-11-10,2020-11-06,122.500,-35.0,2020-11-06
115,Purchase,977810KA7,WOLFFORTH TEX TAX WTRWKS SWR SYS REV C 2021,2021-01-05,2020-11-17,122.096,315.0,2020-11-17


In [95]:
# create sale p&l
x = trades.loc[(trades.action == 'Sale')]

In [96]:
# adjust price for p&l calc
trades['index'] = trades.price/100

In [97]:
x['cost'] = trades.loc[trades['action']=='Purchase', 'price']

<ipython-input-97-c7b6d3ce82af>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x['cost'] = trades.loc[trades['action']=='Purchase', 'price']


In [98]:
x

,action,cusip,description,effective_date,entry_date,price,quantity,transaction_date,cost
0,Sale,01639PBJ9,ALISO VIEJO CALIF CTFS PARTN COPS ...,2020-09-03,2020-09-01,104.738,-150.0,2020-09-01,NaN
2,Sale,64580CFQ5,NEW JERSEY INFRASTRUCTURE BK ENVIRONMENT REF B...,2020-09-04,2020-09-02,122.229,-70.0,2020-09-02,NaN
3,Sale,054231QS7,AVON OHIO LOC SCH DIST GO UNLTD TAX IMPT BDS ...,2020-09-10,2020-09-02,109.422,-115.0,2020-09-02,NaN
6,Sale,306477MG2,FALLBROOK CALIF UN HIGH SCH DI 2016 ELECTION-S...,2020-09-08,2020-09-03,116.678,-25.0,2020-09-03,NaN
7,Sale,306477MG2,FALLBROOK CALIF UN HIGH SCH DI 2016 ELECTION-S...,2020-09-08,2020-09-03,116.678,-35.0,2020-09-03,NaN
...,...,...,...,...,...,...,...,...,...
350,Sale,368414AV0,GEAUGA CNTY OHIO SPL OBLIG SAL BDS ...,2021-07-20,2021-07-16,97.418,-70.0,2021-07-16,NaN
351,Sale,800851TV9,SANGER CALIF UNI SCH DIST ELECTION OF 2018 GO ...,2021-07-20,2021-07-16,98.205,-75.0,2021-07-16,NaN
352,Sale,800851TV9,SANGER CALIF UNI SCH DIST ELECTION OF 2018 GO ...,2021-07-20,2021-07-16,98.250,-75.0,2021-07-16,NaN
353,Sale,018112SY9,ALLEN TEX WTRWKS & SWR REV SYS BDS ...,2021-07-29,2021-07-27,111.564,-100.0,2021-07-27,NaN


In [99]:
trades.loc[trades['action']=='Purchase', 'price']

1      121.058
4      122.413
5      133.098
8      106.219
10     121.390
        ...   
331     96.777
333    108.578
335    111.640
337    111.640
338    109.604
Name: price, Length: 138, dtype: float64

In [101]:
df.loc[df['B'] == 3, 'A']

NameError: name 'df' is not defined

In [ ]:
dff = df.loc[df['Behavior']==1]

In [ ]:
df.at[4, 'B'] =

In [107]:
trades.loc[trades['action']=='Purchase']['price']

1      121.058
4      122.413
5      133.098
8      106.219
10     121.390
        ...   
331     96.777
333    108.578
335    111.640
337    111.640
338    109.604
Name: price, Length: 138, dtype: float64